In [ ]:
import warnings
warnings.simplefilter(action='ignore',category=FutureWarning)
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

#Loading Data

In [ ]:
data=pd.read_csv('/content/sales_data_sample.csv',encoding=('ISO-8859-1'),index_col=0)
data.head()

#EDA

In [ ]:
data.info()

In [ ]:
data.drop(['ORDERLINENUMBER','STATUS','PRODUCTCODE','PHONE','STATE',
           'POSTALCODE', 'TERRITORY', 'CONTACTFIRSTNAME', 'CONTACTLASTNAME'], axis=1, inplace=True)
data.head(10)

In [ ]:
data["ADDRESS"] = data["ADDRESSLINE1"].fillna(' ') + "  " + data["ADDRESSLINE2"].fillna('')
data.drop(['ADDRESSLINE1','ADDRESSLINE2'], axis=1, inplace=True)
data.head(20)

In [ ]:
data['ORDERDATE'] = pd.to_datetime(data['ORDERDATE'])
data['ORDERDATE'] = pd.to_datetime(data['ORDERDATE']).astype(np.int64)
data.head(20)

In [ ]:
nan_df=data[data.isna().any(axis=1)]
nan_df.head(20)

In [ ]:
data.duplicated( keep='first').sum()

In [ ]:
data.describe()

#Q1: What is monthly sales of each financial year?

In [ ]:
monthly_sales=data.groupby(['YEAR_ID','MONTH_ID'])['SALES'].sum().reset_index()
monthly_sales.head(20)

In [ ]:
plt.figure(figsize=(10,6))
plt.title("MONTHLY SALES")
a= sns.lineplot(x='MONTH_ID',y='SALES',data=data,hue='YEAR_ID',palette='dark',errorbar=None,)
sns.set_style('whitegrid')

plt.xlabel('MONTHS')
plt.ylabel('SALES')
plt.legend(prop={'size': 15})  # adjust the legend size

a.set_xticks([0,1,2,3,4,5,6,7,8,9,10,11,12])    # to specifically adjust x-axis
plt.show()

Observations:

1. The sales data from 2003 highlights May (5th) as the month with the lowest sales, while July (7th) emerged with the highest monthly sales.

2. Examining the sales records for 2004 reveals that April (4th) experienced the lowest sales, whereas May (5th) stood out with the highest monthly sales.

3. Turning to 2005, January (1st) recorded the least sales, contrasted with April (4th) securing the position for the month with the highest monthly sales.

4. These observations suggest a recurring pattern, indicating that sales tend to peak during the summer months, while just before this period, the lowest sales figures are observed.

#Q2: What is the best year according to sales?


In [ ]:
yearly_sales=data.groupby(['YEAR_ID'])['SALES'].sum().reset_index()
yearly_sales

In [ ]:
plt.figure(figsize=(7,5))
plt.title("YEARLY SALES") # add title
plt.xlabel('YEARS')
plt.ylabel('SALES (IN MILLIONS)')

z= sns.barplot(x='YEAR_ID',y='SALES',data=yearly_sales,hue='YEAR_ID',palette='mako',edgecolor='black', dodge=False, width= 0.3)
sns.set_style('darkgrid')

tb = ['3.51M','4.72M','1.79M']
z.bar_label(z.containers[0], labels=tb, padding=3) # labelling the first bar with a distance of 3 from the bar
z.bar_label(z.containers[1], labels=tb, padding=3)
z.bar_label(z.containers[2], labels=tb, padding=3)

plt.legend(prop={'size': 12})
plt.show()


Observations:

1.The sales data for 2003 indicates a total of approximately 3.5 million units.

2.Moving into 2004, there was a noticeable increase with sales reaching around 4.7 million units.

3.By 2005, within just 5 months, sales had already reached an impressive 1.8 million units.

4.These figures illustrate a consistent and progressive growth trend in sales over the specified years

#Q3: Which quarter is the best for each product line?

In [ ]:
quarter = data.groupby(['PRODUCTLINE']).QTR_ID.value_counts().reset_index(name='COUNTS')
quarter.head(10)

In [ ]:
# Set the width and height of the figure
plt.figure(figsize=(9,6))

kwargs  =   {'edgecolor':"black", 'linewidth':1} #applying colors to the edge dots

sns.stripplot(x='PRODUCTLINE',y='COUNTS',data=quarter,hue='QTR_ID',palette='bright',size=14,**kwargs)
sns.set_style('whitegrid') # Setting the background

plt.title("PRODUCTS SOLD ACCORDING TO QUARTER") # add title
plt.xlabel('PRODUCTS')
plt.ylabel('NUMBER OF ITEMS SOLD')
plt.show()

Observations:

Notably, during the 4th quarter of each year, all products experienced their peak sales.

This surge in sales during the final quarter may be attributed to the anticipation and celebration of Christmas holidays, suggesting a seasonal influence on consumer purchasing behavior.

#Q4: What product sold the most? Why do you think it sold the most?

In [ ]:
products = data.groupby(['PRODUCTLINE'])['SALES'].sum().reset_index()
price = data.groupby(['PRODUCTLINE'])['MSRP'].min().reset_index()   # MSRP = Manufacturer’s Suggested Retail Price
result=pd.merge(products,price)
result

In [ ]:
# Set the width and height of the figure
plt.figure(figsize=(10,6))

ax = sns.barplot(x='PRODUCTLINE',y='SALES',data=result,hue='PRODUCTLINE',palette='inferno',  # barplot
                 edgecolor='black',dodge=False, width= 0.4)
ax2 = plt.twinx()
sns.lineplot(x='PRODUCTLINE',y='MSRP',data=result,ax=ax2,errorbar=None,color= 'b')  # lineplot

tb = ['3.91M','1.16M','975K','714K','226K','1.12M','1.90M']
ax.bar_label(ax.containers[0], labels=tb, padding=3) # labelling the first bar with a distance of 3 from the bar
ax.bar_label(ax.containers[1], labels=tb, padding=3)
ax.bar_label(ax.containers[2], labels=tb, padding=3)
ax.bar_label(ax.containers[3], labels=tb, padding=3)
ax.bar_label(ax.containers[4], labels=tb, padding=3)
ax.bar_label(ax.containers[5], labels=tb, padding=3)
ax.bar_label(ax.containers[6], labels=tb, padding=3)

sns.set_style('white')  # background
plt.title("PRODUCTS SOLD Vs SALES / MSRP") # add title
ax.set(xlabel='PRODUCTS')
plt.show()

Observations:

The depicted graph reveals a clear trend wherein products with lower prices enjoyed higher sales, contrasting with products commanding higher prices, which exhibited lower popularity.

The inverse relationship between product price and popularity suggests that affordability plays a significant role in consumer preferences, influencing the purchasing patterns evident in the data.

#Q5: Who was the top customer?

In [ ]:
top_customer = data.groupby(['CUSTOMERNAME']).sum().sort_values('SALES', ascending = False).head(5)
top_customer = top_customer[['SALES']].round(3)
top_customer.reset_index(inplace = True)
top_customer.head()

In [ ]:
plt.figure(figsize=(8,6))

# main plotting of the bar chart
z= sns.barplot(x='CUSTOMERNAME',y='SALES',data=top_customer,hue='CUSTOMERNAME',palette='icefire',
               edgecolor='black', dodge=False, width= 0.3)
sns.set_style('whitegrid')

plt.title("TOP 5 CUSTOMERS") # add title
z.set(xlabel='CUSTOMERS', ylabel='SALES')

# to add labels to each bar in the bar chart
tb = ['912K','654K','200K','197K','180K']
z.bar_label(z.containers[0], labels=tb, padding=3) # labelling the first bar with a distance of 3 from the bar
z.bar_label(z.containers[1], labels=tb, padding=3)
z.bar_label(z.containers[2], labels=tb, padding=3)
z.bar_label(z.containers[3], labels=tb, padding=3)
z.bar_label(z.containers[4], labels=tb, padding=3)
# setting the ticks on both axis
plt.xticks(rotation=65)
z.set_yticklabels(["0","200K","400K","600K","800K"])

# adjust the legend size
plt.legend(prop={'size': 9})
plt.legend(loc='upper right')
plt.show()

Observations:

Evidently, the Euro Shopping Channel stands out as the top customer, consistently making the most significant number of product purchases.

The data underscores the notable preference and substantial buying activity from the Euro Shopping Channel, making it a key contributor to the overall sales figures.

#Q6: Which country has the best sales?

In [ ]:
country = data.groupby(['COUNTRY'])['SALES'].sum().reset_index()
# country
short_form = ['AUS','AUT','BEL','CAN','DNK','FIN','FRA','DEU','IRL','ITA','JPN','NOR','PHL','SGP','ESP','SWE','CHE','GBR','USA']
country['iso_alpha'] = short_form
country

In [ ]:
# Set the width and height of the figure
plt.figure(figsize=(8,6))

# Choropleth map using Plotly
fig = px.choropleth(country,locations = 'iso_alpha', color = "SALES", scope = "world", hover_name = "COUNTRY")

# updating the Map by adding the title
fig.update_layout(
    title=dict(
        text="Countries according to their Sales ",
        xanchor='center',
        x=0.5,
        yref='paper',
        yanchor='bottom',
        y=0.9,
        pad={'b': 60}
    ),

)
fig.show()

Observations:

1.The data highlights that the United States achieved the highest sales globally, establishing itself as the top-performing market.

2.Following closely, Spain and France secured the second and third positions, respectively, in terms of sales performance on the international scale.

3.These observations emphasize the strong market presence of the USA, with Spain and France demonstrating commendable sales performance in the global context.

#Q7: Which city has the best sales? Why?

In [ ]:
cities = data.groupby(['CITY'])['SALES'].sum().reset_index()
cities

In [ ]:
cities = pd.read_csv('/content/cities.csv', encoding=('ISO-8859-1'),index_col=0)
cities       # Country names, latitudes, and Longitudes were added.

In [ ]:
# Set the width and height of the figure
plt.figure(figsize=(16,16))

# Plotting Scatter map using Plotly
fig = px.scatter_geo(cities, lat='latitude', lon='longitude', color='iso_alpha',
                     hover_name='CITY', size='SALES',
                    )

# Adding title and centering it
fig.update_layout(
    title=dict(
        text="Cities according to their Sales ",
        xanchor='center',
        x=0.5,
        yref='paper',
        yanchor='bottom',
        y=0.9,
        pad={'b': 60}
    ),

)
fig.show()

Observations:

1.Clearly, the City of Madrid emerges as the global leader with the highest sales, showcasing a remarkable performance in the dataset.

2.Following suit, San Rafael and New York City secure the second and third positions, respectively, underscoring their noteworthy contributions to the overall sales figures on a global scale.

3.These observations highlight the standout sales achievements of Madrid, San Rafael, and New York City in the dataset.